# The Tammes problem

**Prompt for the search setup**

Act as a research mathematician and optimization specialist specializing in
constructing collections of 3d points with a certain optimal behaviour.

GOAL:
For a given natural number n, your task is to provide a set consisting of n
three-dimensional points x_1, ..., x_n, on the unit sphere so that the
minimum among all pairwise distances \|x_i - x_j\| is as large as possible.
That is, the collection x_1, ..., x_n maximizes the quantity:

Q(x_1, ..., x_n) := \min_(1 \leq i < j \leq n) \|x_i - x_j\|

where \|x_i - x_j\| denotes the standard Euclidean norm in R^3.

The end points are given as a python numpy array of shape [n, 3].

Specifically, the Python function you have to provide is called get_positions.
For example, here is an starting implementation to get you going:

def get_positions(n: int) -> np.ndarray:
  """Generate positions to minimize the area of the union."""
  variable_name = "suggested_positions_" + str(n)
  best_positions = np.random.randn(n, 3)

  if np.random.rand() < 0.5 and variable_name in globals():
    best_positions = globals()[variable_name]
  curr_positions = best_positions.copy()
  best_score = get_score(best_positions)
  start_time = time.time()
  while time.time() - start_time < 300:
    random_index = np.random.randint(0, len(curr_positions))
    curr_positions[random_index:] += 1e-1 * np.random.randint(-10, 10)
    curr_score = get_score(curr_positions)
    if curr_score > best_score:
      best_score = curr_score
      best_positions = curr_positions.copy()
  return best_positions

EVALUATION:
Your proposed construction will be evaluated by an external function called
get_score(x) - here x is your configuration as a numpy array of shape [n, 3].
The function get_score(x) computes and returns Q(x_1, ..., x_n) or in case of an
error or any problem with your configuration returns a very large negative
value. The higher score your configuration has, the better.

You don't have to implement this scoring function get_score; it's already found
elsewhere in the codebase.

HINTS:
As a starting point one can try to construct the points uniformly on the sphere,
e.g. not concentrating all the points in one hemisphere.

The previous solution provided in this prompt is not optimal, much
better algorithms are possible. The patterns you have to discover are not
hard, you can definitely improve it, it is not beyond your capabilities. DO NOT
go for the same solution as the previous one. Always try to find a better
pattern, don't be scared of the difficult sounding problem, once you see the
solution you'll realise it wasn't hard at all. Good luck, I believe in you, but
you also have to believe in yourself!

You may code up any search method you want, and you are allowed to call the
get_score() function as many times as you want. You have access to it,
you don't need to code up the get_score() function.
You want the score it gives you to be as high as possible!

Your task is to write a search function that searches for the best list.
Your function will have 1000 seconds to run, and after that it has to have
returned the best construction it found. If after 1000 seconds it has not
returned anything, it will be terminated with negative infinity points. You can
use your time best if you have an outer loop of the form
"while time.time() - start_time < 1000:" or similar, just don't forget to define
the "start_time" variable early in your program. You'll have to balance your
resources carefully!

In [ ]:
# @title Initial program

import numpy as np


def get_positions(n: int) -> np.ndarray:
  """Generate positions to minimize the area of the union."""
  variable_name = f"suggested_positions_{n}"
  best_positions = np.random.randn(n, 3)

  if np.random.rand() < 0.5 and variable_name in globals():
    best_positions = globals()[variable_name]

  curr_positions = best_positions.copy()
  best_score = get_score(best_positions)

  start_time = time.time()

  while time.time() - start_time < 300:
    random_index = np.random.randint(0, len(curr_positions))
    curr_positions[random_index:] += 1e-1 * np.random.randint(-10, 10)

    curr_score = get_score(curr_positions)
    if curr_score > best_score:
      best_score = curr_score
      best_positions = curr_positions.copy()
  return best_positions

In [ ]:
# @title Evaluation

import dataclasses
import enum
import scipy

NEGATIVE_INFINITY = -1e10


class ErrorMessage(enum.Enum):
  NO_ERROR = 0
  WRONG_OUTPUT_SHAPE = 1
  FUNCTION_IS_TOO_LARGE = 2
  ZERO_FUNCTION_SCORE = 3
  STOCHASTIC_FUNCTION = 4
  OTHER_ERROR = 5
  NO_POLYGON_SET = 6
  POINTS_NOT_FLOATS = 7


@dataclasses.dataclass
class ConfigurationEvaluationLog:
  energy: float
  error_message: ErrorMessage


def get_energy(points: np.ndarray | list[float]) -> ConfigurationEvaluationLog:
  """Calculates the energy of the given configuration of points."""
  points_array = np.array(points)
  n = points_array.shape[0]
  if n <= 1:
    # No pairs to calculate energy for
    return ConfigurationEvaluationLog(
        energy=0.0,
        error_message=ErrorMessage.NO_POLYGON_SET,
    )

  points_array /= np.linalg.norm(points_array, axis=1)[..., None]

  pairwise_distances = scipy.spatial.distance.pdist(
      points_array, metric="euclidean"
  )

  if np.any(pairwise_distances < 1e-12):
    print(
        "Warning: Some points are coincident or extremely close. Potential"
        " energy approaches infinity."
    )
    pairwise_distances[pairwise_distances < 1e-12] = 1e-12

  total_energy = np.min(pairwise_distances)

  return ConfigurationEvaluationLog(
      energy=total_energy,
      error_message=ErrorMessage.NO_ERROR,
  )


def get_score(x: np.ndarray | list[float]) -> float:
  """Computes the area of a given configuration of triangles."""
  config_eval_log = get_energy(x)
  if config_eval_log.error_message != ErrorMessage.NO_ERROR:
    return NEGATIVE_INFINITY
  return config_eval_log.energy


In [ ]:
# @title Constructions obtained by AlphaEvolve


tammes_results = {
    3: np.array([
        [-0.18199808250933916, 0.9507997123560856, 0.2507121954483044],
        [-0.6632346671845527, -0.7128970326139821, 0.22781044123300126],
        [0.8452327496938922, -0.23790267974210358, -0.47852263668130535],
    ]),
    7: np.array([
        [-0.013687589801812722, -0.5465509278303369, 0.8373139991503875],
        [-0.6529881328921786, 0.1330139607240133, -0.7455962610920855],
        [0.08416314818582643, 0.9799249834456082, -0.18073071489532525],
        [-0.7946294471363553, 0.3712803076481891, 0.4803279867928128],
        [0.5970786036004467, 0.015239331227594901, -0.802037969117595],
        [0.8895809106440732, 0.21260297663630323, 0.40428427837728903],
        [-0.10358032343952527, -0.9284371549626664, -0.35675701237816343],
    ]),
    12: np.array([
        [0.5441432905753543, 0.7208044665521575, -0.4293588246679989],
        [0.2730778863314617, -0.27801095678154397, 0.9209442849087842],
        [-0.434042989879008, 0.8967596135660977, -0.08619094159872434],
        [0.4340429898790079, -0.8967596135660977, 0.08619094159872434],
        [0.6015706228759634, -0.2794577334582553, -0.7483422752344012],
        [-0.34112361433338745, -0.7216986237124957, -0.6023170064638853],
        [-0.5441432905753539, -0.7208044665521577, 0.42935882466799885],
        [0.3411236143333873, 0.7216986237124958, 0.6023170064638853],
        [0.9811680262280312, -0.005243674905784337, 0.1930849765809655],
        [-0.2730778863314617, 0.27801095678154414, -0.9209442849087841],
        [-0.6015706228759634, 0.27945773345825525, 0.7483422752344012],
        [-0.9811680262280312, 0.005243674905784244, -0.19308497658096538],
    ]),
    25: np.array([
        [0.05431862619639665, 0.9972541692577231, 0.05033496544376253],
        [0.4638798590227872, -0.4374722232046733, -0.7703463703538556],
        [0.9580893475590049, 0.20261288659514956, 0.2025162222626664],
        [0.5074464484148833, 0.6961883864807787, 0.5077596208053681],
        [-0.7913175109299349, 0.6001188525646822, -0.11693570751529134],
        [-0.9890422477345545, -0.03059225410035834, 0.14442834273521046],
        [0.8608413235646555, 0.11389439742561709, -0.49596399252214646],
        [0.6663123898735528, 0.7224778661175517, -0.184535991262474],
        [-0.26263204924618394, 0.7916082244461665, -0.5517071919940304],
        [0.3483012830255846, 0.5128545844448665, -0.7846441177098156],
        [-0.20860407283894275, -0.6620174508622869, -0.7198730690537081],
        [0.8380092492751652, -0.51803785168941, -0.17139802316915828],
        [-0.11995476608332797, -0.31278451308896815, 0.9422190310461742],
        [0.2645882360678135, -0.9283135378474751, -0.26120306426921475],
        [-0.4483015039344377, 0.749270744640592, 0.48746190907172887],
        [-0.42380391615067714, -0.9016377280467673, -0.08625339424053677],
        [-0.06712855045030314, -0.023171263230577475, -0.9974752379256041],
        [-0.8055762587789842, -0.4093019542744217, -0.42839094472081735],
        [-0.6663992854068946, 0.22840517711874278, -0.7097485945562245],
        [-0.027961066504659203, -0.8638994923488444, 0.5028875081758654],
        [-0.01447239960083964, 0.38979426239326576, 0.9207882398548993],
        [-0.6498119440092385, 0.12045243991635855, 0.7504902711835314],
        [-0.6644937544997269, -0.5476746208092873, 0.5084295034242802],
        [0.5999281251639402, 0.054097281500186674, 0.7982228565830279],
        [0.6288372568317191, -0.5922051921457001, 0.503841954203928],
    ]),
    32: np.array([
        [-0.5826711047114449, 0.51534321614103, -0.6284232278582359],
        [0.5589854932796052, 0.7603723915094577, 0.33070990994094],
        [-0.05716604185639478, 0.8786193662736048, 0.47408865507143483],
        [-0.7442831433667922, -0.10569284713727217, -0.6594479695655029],
        [0.021631976593499923, -0.7613356621931889, 0.6479969653181386],
        [0.9375415212740825, 0.3475021635709402, 0.01606680430560823],
        [-0.22188310992275131, 0.10729574738485287, -0.9691519530621264],
        [-0.8599678297356086, 0.012782173676316047, 0.5101881494663864],
        [0.1595509853235646, 0.9795925010867624, -0.12223753472997134],
        [0.5956842248241268, -0.4728410927671359, 0.6492931582008702],
        [-0.6428271540499053, 0.6150639835865269, 0.4565846538274933],
        [-0.4356104588280197, -0.34833489241332793, 0.83000381377861],
        [-0.6831951592358944, -0.6550748692727308, 0.3226783073650221],
        [-0.9108492657580384, 0.4033423111426398, -0.08757051507303318],
        [0.15430318103555973, -0.14120904222276018, 0.9778806341864216],
        [-0.9690166499898382, -0.236296636907, -0.07190710277095788],
        [0.7843673998034678, 0.13965176169497268, 0.6043717130880941],
        [0.6112456732429219, 0.65519423754719, -0.4439585994513043],
        [0.9482192868173898, -0.28128814356363346, 0.14750309962182329],
        [0.2847016910525477, 0.47138050320504715, 0.834712745985094],
        [0.3594136423433089, -0.40242745435426514, -0.8419465408679235],
        [-0.3248341771021116, 0.28025392061244186, 0.903294247389935],
        [0.401595288044627, 0.23723801612016404, -0.8845560176312872],
        [0.7083174496207919, -0.6151726333203749, -0.34619217463191276],
        [0.13289073503876175, -0.8734825848575039, -0.4683676189613361],
        [-0.18460547404722116, -0.9795509344470595, 0.08000491094735],
        [0.02286484412229933, 0.7198572612870486, -0.6937454304538322],
        [0.4427148244310514, -0.8792590070618551, 0.1758043876858138],
        [-0.4739970127754188, 0.8740484938693023, -0.10661173596177752],
        [-0.620429140374901, -0.7188658780259005, -0.3135275604850597],
        [0.8599678255665034, -0.012782187296056891, -0.5101881561525596],
        [-0.27266031242042316, -0.5139241850487568, -0.8133500390685233],
    ]),
    50: np.array([
        [0.7322901985853106, -0.580873433242494, -0.35543933323278176],
        [0.8668577253808234, 0.29709842023339167, -0.40036260145324254],
        [0.09188746042046939, 0.8178982615454883, 0.5679781055448748],
        [0.41753906891157594, 0.7210915371304273, -0.552890695356082],
        [-0.7479003654736476, 0.6359554715278511, 0.1902779061220013],
        [0.505801994465565, -0.03341820449736239, 0.862002068444635],
        [-0.9373130668768048, -0.048481761768592896, -0.3450995993011466],
        [0.31438814393675263, -0.8388229551939425, 0.44445038507318796],
        [-0.7527465493975529, -0.5247635341322129, -0.3974869376660664],
        [0.8097328358764708, -0.5667802456649191, 0.1519634417465671],
        [-0.35039337926840286, -0.5149747971085981, 0.7823205469037788],
        [0.7608528722093818, -0.14893360140102668, -0.6316024772152611],
        [0.7031992081066337, 0.6996024139865048, -0.1267569961085325],
        [0.16052783862765743, -0.5355084499435196, 0.8291329887687631],
        [-0.0829631173116482, -0.4823522455455945, -0.8720398112374551],
        [-0.19797421324956202, -0.872482290982598, 0.44674474010331017],
        [0.1745502547985098, -0.8642704767830905, -0.4717719274298806],
        [0.9442997680276304, 0.31119178880473675, 0.1070402666451824],
        [-0.48438920248932094, -0.027923825885594503, 0.8744068620840476],
        [-0.6555042626238464, -0.6462434634107526, 0.39074742184797967],
        [-0.24842750976476122, 0.11160835876493462, -0.9621992239894382],
        [0.27694916860990354, 0.9608364341162783, -0.00961794577849917],
        [-0.18716646366976272, 0.3898929511567737, 0.9016386202440627],
        [0.7473571578426565, 0.3221204451031153, 0.5811159070855981],
        [0.07581470606828906, 0.49749244658639424, -0.8641489431418999],
        [-0.5291907885285076, -0.8460019631842343, -0.06509829202827139],
        [-0.7392775647612096, 0.31816313051783696, 0.5934988665699584],
        [0.6257898735970058, -0.47103814864058924, 0.6216993619336356],
        [0.31930600444180995, 0.44433577480136294, 0.8370241303326204],
        [0.22612525745167922, -0.08431303325833629, -0.970442517805764],
        [-0.5077589635060772, 0.8333556296598351, -0.21840153271782226],
        [0.5374730743178054, 0.2839261574194893, -0.7940457364134451],
        [0.557377500854481, 0.7465391064802271, 0.36333136946445627],
        [-0.3815346648627622, 0.6327060625971085, -0.6738800619256544],
        [-0.038369176293852246, 0.9104979022134303, -0.4117297370551152],
        [0.9157376406447653, -0.12074800444650675, 0.38320293961365326],
        [-0.32822982500784675, -0.7798822686271368, -0.5329623148556649],
        [-0.5447609503606935, -0.2721612378041721, -0.7931984415006217],
        [-0.20273714329317305, 0.9637334138376233, 0.17353834671969107],
        [-0.8296725523645236, -0.18295159099343192, 0.5274202984394031],
        [0.41590583225349836, -0.5092324165254462, -0.7534618003967949],
        [0.9768607557168405, -0.15740225918673897, -0.14480190863118433],
        [-0.9768864682708631, 0.15719194937515044, 0.1448568920035228],
        [-0.6788742016826994, 0.21503580197205505, -0.7020608393578954],
        [0.012084482910697728, -0.0735915541532142, 0.9972152467898271],
        [-0.8469181007296209, 0.4526330520844084, -0.279021595610958],
        [-0.04060697495737542, -0.9988523159782288, -0.025399300182086026],
        [0.4600635849276428, -0.886714806932079, -0.04558891302643101],
        [-0.4105165258632419, 0.7118757572331307, 0.569832508950596],
        [-0.9286899228617197, -0.3662744606040965, 0.05812096596127289],
    ]),
    100: np.array([
        [-0.06014714583127452, 0.9440337356722528, 0.3243187117035381],
        [0.671293679327071, 0.06351926167619382, -0.7384646907548357],
        [-0.47169411313805876, -0.8650249115636134, 0.17098703461158396],
        [0.09206227898577943, 0.8301325925315864, -0.5499130982298308],
        [-0.2029297475799889, -0.09097055373280027, 0.9749583970101852],
        [0.25745015382478925, 0.5514870183806855, -0.7934617110189842],
        [0.9731998911315818, 0.11329692604629937, 0.2001144133987772],
        [0.45057396229103924, -0.372981171460252, 0.8110907164069204],
        [0.09059733717893553, -0.03467303780110149, -0.9952838303447563],
        [0.23232342174545198, -0.8166595498464693, -0.5282925395583843],
        [0.19732682258880083, 0.5820034064079654, 0.7888815880831118],
        [0.8367855212819958, 0.5467256389392681, 0.029682774453674145],
        [-0.8353891961907081, -0.4540015201722646, -0.3098507875883414],
        [-0.15013997540100266, -0.9874511342117241, 0.04897188306126165],
        [-0.6197238609619068, 0.7769032851834115, 0.1111918235559176],
        [0.20052577921081322, -0.6315240666747036, 0.7489771459013559],
        [-0.8589370913592321, 0.5012955914315613, 0.10454569861368526],
        [0.42114710891298496, -0.1519002477441882, -0.8941819878467072],
        [0.45436199951973055, 0.7140394380684858, -0.532637638808293],
        [-0.662367603228168, 0.18700338305402317, -0.725464604870646],
        [0.8896503349685255, -0.4563968392271453, -0.01497353110865085],
        [-0.43498271532091126, 0.5947470832553357, 0.6760665235990596],
        [-0.3840676702576768, 0.3969736549804964, -0.8336089862245165],
        [0.5655622859162492, -0.7009065280341029, -0.43459100279270163],
        [0.9898898139187919, -0.11271225565299983, -0.08610518988674482],
        [0.34262067517027994, 0.9210514118203006, 0.18513608435335552],
        [0.21550705492721495, -0.8591842458523565, 0.46406803483510456],
        [-0.4045980668552278, 0.2245806350688585, 0.8864896743048285],
        [-0.4109474212707657, -0.8920838109417472, -0.18790607016953165],
        [0.5101139984768114, 0.6130072916288933, 0.603328906126508],
        [0.8339036217192887, 0.4107012106188598, 0.3686858625980344],
        [-0.6311295498034782, -0.43155215405501096, 0.6445449787992604],
        [0.6745455389494994, 0.29546802310306763, 0.6765256559857096],
        [-0.39817196251249937, 0.8269979529841355, 0.39690486773154404],
        [0.6058059314329635, -0.043052753307181946, 0.7944467470342557],
        [0.3750367364712548, 0.27782760070146767, 0.8843976880258431],
        [0.7372780495327647, -0.3305018704592488, 0.5892279620826733],
        [-0.5393613458166612, -0.11066690994485467, 0.8347707312083346],
        [0.4144390591613204, -0.5456707933018129, -0.7283430864495453],
        [0.36562920327107823, 0.2086244852702415, -0.9070783372240148],
        [0.14185617723041746, -0.3862637894628125, -0.9114148945082039],
        [-0.6716580589777514, -0.39020928466504656, -0.6297715188633353],
        [-0.16150602167348369, 0.9369002751168503, -0.31005431693362157],
        [0.5958894612604411, 0.7993740212843453, -0.07692154480011787],
        [-0.25429752629663166, 0.0805703894260692, -0.9637640688815627],
        [-0.7048438452505376, -0.7049318716684029, -0.079159396905982],
        [0.20814581990478728, -0.9711076580871548, 0.11672717790063078],
        [0.22591411290735017, 0.8253407105766581, 0.5174703132104315],
        [-0.15606896652244434, -0.6941682781694627, 0.7026897460984184],
        [0.3717368788162961, -0.9058767242086821, -0.2029754996665007],
        [-0.1006900796644237, 0.6202017478900826, -0.7779532760849192],
        [0.035810224985443916, 0.18503539562505775, 0.9820791873124948],
        [-0.6763538215428772, 0.32126727231547675, 0.6628218824264324],
        [-0.14178374796582388, -0.9036482781586951, 0.40412517639165885],
        [0.8491930754967928, 0.011474199555408886, 0.5279578233844642],
        [0.6340476571132727, 0.7217193079675864, 0.27767752702723014],
        [-0.8625450525801996, 0.26573781084599046, -0.43058036201875394],
        [-0.6378722218407159, 0.5168003615545935, -0.5709959850130558],
        [0.020441114168800085, 0.3202948664352485, -0.9470973336393491],
        [-0.020858070891349646, -0.6671930895655386, -0.7445927223084328],
        [-0.3664914655638298, -0.5500170270813323, -0.7504433859988969],
        [0.5113631251140415, -0.8210536326879696, 0.2537295539023585],
        [-0.9733195471983581, 0.2123556308639098, -0.08691458497843813],
        [-0.8658743178523867, -0.3100225354365179, 0.39261647087861334],
        [0.30563247702521196, 0.9142078971528109, -0.26610657596322007],
        [-0.8009289744695828, -0.01574516747338255, 0.5985523097911641],
        [-0.8939965330483183, 0.25302279895392915, 0.36979678487935336],
        [0.8943318198179481, -0.09227423547062677, -0.4377854057976678],
        [0.9302024253030353, -0.2409378145437127, 0.27689784665694045],
        [-0.14470373337763887, 0.4799841288461982, 0.865260692278535],
        [0.00583018723667858, -0.9599968891397386, -0.27994996295554264],
        [-0.9723589314880321, -0.13834104062915237, -0.1880953610089725],
        [-0.35828315554188594, 0.7508824697275216, -0.5548050983100699],
        [-0.252778342596821, -0.8225504835331945, -0.5094249812811616],
        [-0.5251843700848335, -0.1309476513557018, -0.8408531917184011],
        [-0.11336850042570018, 0.7627846255841042, 0.6366374149260677],
        [0.8350602289258694, 0.2651538079716014, -0.48204550841640864],
        [0.5182671988660326, -0.6380505818776969, 0.5694651574460802],
        [0.9589462311735311, 0.2470814029030425, -0.139186587204174],
        [-0.30428238813028596, 0.951102332103454, 0.05307148142930638],
        [0.736242197467353, 0.5860716921748489, -0.33833030946066955],
        [0.6753760489958585, -0.2959829147835089, -0.6754711737735387],
        [-0.7726280044307197, 0.588015784090621, -0.23933951706664736],
        [-0.33037288135060533, -0.409509190742547, 0.8503857841976683],
        [0.28107700242738, -0.08401984289579517, 0.9560002012061528],
        [-0.7278190235428489, -0.6274329881090239, 0.276780263750354],
        [0.8158194660942523, -0.4426408661257867, -0.37216617575634503],
        [0.7647661724159955, -0.5645557376130713, 0.3104988255313519],
        [-0.9837401068903214, -0.04205793051953623, 0.17460393058529078],
        [0.5825646653860275, 0.415328060056338, -0.6986565774209135],
        [0.6676394702699247, -0.7415183277533319, -0.0663935790841007],
        [-0.5025913004629583, 0.8334074533698089, -0.22985647993172253],
        [-0.20649247832289022, -0.2804128932796697, -0.9374057102869575],
        [-0.5830539295768398, -0.6860411534078978, -0.4351961064115427],
        [-0.6915418572225972, 0.5901790069809498, 0.41648361243643306],
        [-0.9099777097896992, -0.4122085237952054, 0.04499667317115473],
        [0.041446107945015354, -0.3588111340633633, 0.932489565736997],
        [0.050319871121473445, 0.9985472592988209, -0.019268666719097344],
        [-0.8456922415738866, -0.08604673940248006, -0.5266883245145434],
        [-0.46338339917749954, -0.7258695446494257, 0.5083200070006507],
    ]),
    200: np.array([
        [-0.6981240205075624, 0.7030514419395867, 0.13543087527231298],
        [-0.10017979715243779, 0.5454331376516484, 0.832145840940141],
        [-0.9151171779848495, -0.3216642936746839, -0.2430897627046683],
        [0.738195374213234, -0.02092995596980053, 0.6742622089612362],
        [0.6280998559214123, -0.5123701815272371, 0.5856341589023368],
        [-0.10810157335556744, -0.6217921953930976, -0.7756858356230846],
        [-0.8379337453138488, 0.34144691134913757, -0.42577111831879827],
        [-0.404074139421992, 0.28062546901933194, -0.8706166986613895],
        [0.7857716891179611, 0.6133819868050399, -0.07953232577894392],
        [-0.33245021352696436, 0.5153155407199174, -0.7898903398690321],
        [-0.11981006329733626, 0.8701089581704825, -0.4780752552100622],
        [-0.3877129856455086, -0.21606341638193566, -0.8961000172208524],
        [0.5910524311012886, 0.6437215890040432, -0.48608593842999254],
        [0.5144715715201653, 0.5070224306312234, -0.691554232822255],
        [-0.2821752062770619, -0.9403249529550478, 0.19017396197320188],
        [-0.9873232931427489, 0.13447615893127093, -0.08431415952762165],
        [-0.6027348274340498, -0.5522083407747722, -0.5760005869587455],
        [-0.8633239349470596, 0.3958860409462581, 0.31296329805813133],
        [0.6325543297008465, -0.7727247414078626, -0.0526449806996307],
        [-0.20579386906112127, 0.9477262378350112, -0.24385213055446095],
        [0.5553607446280729, -0.12808791016846358, 0.8216860292075241],
        [-0.04530607950478214, 0.3160130919356268, 0.9476724565403342],
        [0.15289396327644, 0.9454237892609816, 0.28774519056455117],
        [-0.2781677768350954, -0.6405451642405973, 0.715768524383831],
        [-0.8716458655228209, -0.17675803854819755, -0.4571543294398146],
        [-0.6689661507939122, -0.7391792563633683, -0.07809171565585282],
        [0.9697006777492029, 0.05960030359339004, 0.23691432920849792],
        [-0.5329921934968455, -0.6686743309986173, 0.5184534315972584],
        [0.5111222089939338, -0.37755450371497623, 0.772144212047011],
        [0.6962147038622283, -0.32274772444497846, -0.6411856146948544],
        [-0.8599328800849474, 0.19297093624140904, 0.4725226550282234],
        [0.429043322092767, 0.8172689488939114, -0.38469896405039034],
        [0.012248192637778677, 0.9987097757633882, -0.04928250776645533],
        [0.3606509907251157, -0.7801064980177815, 0.5112384127189834],
        [0.8852478095927335, -0.4385926414783684, 0.15483155509228294],
        [0.8092624875724691, 0.1960409173329201, 0.5537708776554493],
        [-0.7340880291576061, 0.4068515084235908, 0.5436787797412648],
        [0.7162546166006817, 0.4276305857576524, -0.5514629691309996],
        [-0.7733533596526688, -0.013892613410227347, -0.6338229850727757],
        [-0.470937407052096, 0.6601846131479695, -0.5851275375520403],
        [-0.07157816970200104, -0.7874429787072775, 0.6122173804350232],
        [-0.4942468864159254, -0.8670731215174358, 0.06248373556513295],
        [0.048645353613562106, 0.952493854333158, -0.30064777901953016],
        [0.3695304306041289, 0.2614833499612262, 0.8916690633584762],
        [-0.8352767816519312, -0.2633953733480594, 0.4826339972816135],
        [0.8970470483071803, 0.027989131235187117, -0.44104784508721373],
        [-0.5204278522900293, -0.39032164256414403, -0.7594760469605452],
        [0.7685153421874561, -0.5225363071888794, 0.36924243593052314],
        [-0.3706251387959316, -0.6281338831910411, -0.6841672538786403],
        [-0.8201603009469852, -0.5482157090389134, -0.16369672297731616],
        [0.28752396661513724, 0.9357518134474151, -0.20420213576692],
        [0.3642216501559143, 0.006440157095544658, 0.9312900267555148],
        [0.3071087527662425, -0.292592433522377, 0.9055793073054488],
        [0.7619164547180023, 0.43797311835670216, 0.4771402976346232],
        [0.17157333848210746, -0.9484702786583119, -0.2663957958075325],
        [-0.08433915371229338, -0.9958603068408163, 0.03389920957794915],
        [0.11925739507549939, 0.6525188441340427, 0.7483293604889356],
        [-0.6797515045143854, -0.6665229500565872, 0.3060801351905964],
        [-0.48980206215385386, -0.8394221110831567, -0.23550893684641555],
        [0.5230927896770986, -0.24514036860194766, -0.8162598440873665],
        [0.19465154768911824, 0.8509821846632001, -0.48779103760532366],
        [0.999084720483965, -0.04274372912586533, 0.001641619898856543],
        [0.5522043433768419, -0.536976979879612, -0.6377508026142512],
        [-0.10008718936702057, 0.7502400430388256, 0.6535460445490519],
        [0.8059600808226473, -0.5846904178286271, -0.09254978886922115],
        [-0.8000238125904209, -0.5932200393201666, 0.08973229205401613],
        [0.3416165717502299, 0.39983967612234916, -0.8505447379791347],
        [-0.24703829400062866, -0.4334136368929718, 0.8666745067512176],
        [0.8753733121674264, -0.4044593515633692, -0.26482861869135005],
        [-0.2698958704894522, 0.9628716786602571, 0.005860846928498874],
        [-0.7800079690808723, 0.5605517740231112, -0.27815333327119385],
        [-0.16238904773280657, 0.3567680079855709, -0.9199708613072699],
        [-0.28467707692848365, -0.4477515419933392, -0.8476305318439403],
        [-0.22975816251028708, 0.7188720680223585, -0.6560747949564548],
        [0.02975133314268655, -0.3561320638191981, 0.9339618896379613],
        [0.09596741241429001, -0.08364628419340865, -0.9918636775762791],
        [-0.3820361746543167, -0.04702087787002251, 0.9229503769433268],
        [0.32486114005068967, -0.6685713264649096, -0.6689376810390617],
        [0.7659682007856159, -0.07420648035741696, -0.6385813289301614],
        [0.5883569088638841, 0.004513447623487152, -0.8085887561564812],
        [0.6638811923838625, -0.6874929789757764, -0.2943045467844389],
        [-0.18722084331127906, -0.7843446955976757, -0.5913981351996327],
        [0.08051762244926214, -0.750809708375378, -0.6555926283023613],
        [0.9025486885001057, 0.30253371884289393, 0.3063971505249814],
        [0.179038193732802, 0.43650698685958333, 0.8817068535560235],
        [0.8583882238394054, -0.23286581043764132, -0.45710302066717035],
        [0.4967638241669312, 0.8650125605262023, -0.0705618390558893],
        [-0.5291438121927683, 0.650615063854322, 0.5447080545612859],
        [0.0029315279456281415, -0.8913076190890385, -0.4533896054142989],
        [-0.9944124051949165, -0.10485825547581064, 0.012194861747111983],
        [-0.660681184018596, 0.7414785304056216, -0.11708954706251962],
        [0.3748144595293869, -0.4562656783151417, -0.8070537477264556],
        [0.5850661979770861, 0.2697557163303044, -0.7648067713427762],
        [-0.5411394957208056, 0.45196875500979733, 0.7091489904568196],
        [-0.8357456483175042, 0.5487815011222624, -0.019185289790643323],
        [-0.5933065855888071, 0.7174012193565988, -0.3651339288023187],
        [-0.7217301902082232, -0.3127521357975914, -0.6174881651465451],
        [0.4166016572105774, -0.8673963318716434, -0.27215191100747094],
        [-0.9240527654034041, -0.3821871170234678, 0.007713256874460411],
        [0.41621326382183926, -0.9091007942104612, -0.01738576960277476],
        [0.7716687191944321, 0.18222813856065814, -0.6093605610256888],
        [0.6398369711051085, 0.34689965835566383, 0.685761822675894],
        [-0.46568700740317054, -0.746450963912033, -0.47533837380404736],
        [-0.0813014920988611, 0.5770980289904964, -0.8126179497880804],
        [0.9584172515383756, -0.19655624576007136, 0.20688647661539536],
        [0.026092166107189688, 0.7532904817235896, -0.6571701826867049],
        [-0.5366568422329863, 0.7801422353939396, 0.32152375688130236],
        [-0.9437295468354263, -0.24305282941353656, 0.22427631293535075],
        [0.34673334858733396, 0.6922436439361439, 0.6329097269022049],
        [-0.32533555783903617, 0.42003920373879566, 0.8471858368315901],
        [0.24885354154643863, 0.9672965076765101, 0.04908544689235395],
        [0.8707754682389139, 0.29025308866518285, -0.39686676408288696],
        [0.32317983357422286, -0.9018702514094059, 0.2866789925921895],
        [0.1525238347270232, 0.16760390047396523, -0.9739842978128944],
        [0.12696553321423437, 0.8225986946063569, 0.5542663087429567],
        [-0.7074518720529926, 0.18975678751033098, 0.6808114352171791],
        [-0.31844661091161486, 0.9084321028307055, 0.2708188888269962],
        [-0.36752189657898054, 0.831565885524662, -0.4164442742631645],
        [-0.3464857080282613, 0.0360760509610073, -0.9373612818327912],
        [0.5847363253377799, -0.6986618945490466, 0.41225597258936947],
        [-0.4738271541531002, -0.8218852602226748, 0.31621582347485483],
        [0.5752915201845127, 0.726213657068322, 0.3763686903705546],
        [0.19900309034748948, -0.31186420638116347, -0.929052467199994],
        [-0.5018650650229926, 0.8613683448487948, 0.07858772806179881],
        [0.715458641607688, -0.27602893395419953, 0.6418153626776083],
        [0.8858480453031767, -0.03309422553930154, 0.4627937044391324],
        [-0.8319157625080278, -0.4580976795070696, 0.31314961299820415],
        [-0.3041176601482171, -0.9481002122379395, -0.09283553382378763],
        [0.9672453415366267, 0.13328457691808532, -0.21603627202731893],
        [0.7637146447421811, 0.5877911724261744, 0.26691099457344836],
        [-0.18010420469673766, -0.20320537686669987, 0.9624292442891675],
        [0.08033644322497253, -0.9696281330960468, 0.2310137212345533],
        [0.13431038330752507, -0.5482695815169699, -0.8254460533063276],
        [-0.49437135657853126, -0.4995423233543552, 0.7113750269530381],
        [0.16963168694183908, -0.7191398086073685, 0.6738419892388894],
        [-0.3103179923987643, -0.7992395752753673, 0.5147026762192433],
        [0.14062715940057074, -0.1114432460683846, 0.9837705042054615],
        [-0.27345880881272727, 0.18627881363257534, 0.9436739285761605],
        [0.9611361395195863, -0.12763238165128765, -0.24480052382175652],
        [0.6317171134482632, 0.7345312361382339, -0.2477848900030675],
        [0.415325053729251, 0.4844777532631884, 0.7699262343483614],
        [0.8576363613491084, -0.28714338816942925, 0.42662459648084305],
        [-0.11755345249022792, -0.91441305681002, 0.38733699454477977],
        [0.4477075777216054, 0.8747787521389739, 0.18525673984185437],
        [0.3469894880101847, 0.838484637276744, 0.4201687854437989],
        [0.40589964662375594, -0.5993214254011054, 0.6899705109103559],
        [-0.6079677032574978, -0.14765550778652065, -0.7801109682706093],
        [-0.9142934556276895, 0.3628138605804659, -0.18009325241961907],
        [0.20420660238142696, -0.5173243901244667, 0.8310686728100045],
        [-0.980170634876228, 0.09663681218855129, 0.1729937948493146],
        [0.48812125448202315, -0.7298602744669186, -0.4785829297812142],
        [0.12429731773240035, -0.8747105020932681, 0.4684353897094813],
        [-0.7312492336213648, 0.2407701337121739, -0.6382039650773008],
        [0.15254028784489157, 0.1504967980035789, 0.9767713009578382],
        [0.7422947924545777, -0.6534574662454629, 0.14829626058291273],
        [-0.09997051967835308, 0.11575117785569863, -0.9882345673069014],
        [-0.2754585213367037, -0.8866620639398263, -0.37142023018807746],
        [-0.09019825633499179, 0.8960333387185639, 0.4347281109601615],
        [-0.6056395656316361, -0.02392409759499614, 0.7953793774644828],
        [0.9551218068633937, -0.2925770881446356, -0.04627074180097395],
        [-0.9402475738575558, 0.3321470418933315, 0.07491890560125505],
        [-0.04354560496662479, -0.586393084903646, 0.8088553209723457],
        [-0.7117745140482149, 0.5968492814627343, 0.37033495158956475],
        [-0.5844464687973449, 0.10829946106778776, -0.8041725883432866],
        [-0.5595628560345793, 0.4170848285846828, -0.7161910750008761],
        [-0.6638629937219432, -0.6723781544171195, -0.32740425016967584],
        [0.0901017992252381, 0.409244181896207, -0.9079652335637522],
        [0.40043999609838116, 0.15406706892127092, -0.9032780013920061],
        [-0.7836850794691261, -0.4614813742265656, -0.41577955392182303],
        [-0.08358634697351178, 0.978305368112065, 0.1895571927486291],
        [-0.6787422297691562, 0.52454342245369, -0.5139680763321434],
        [-0.6674893540636536, -0.2584584070531971, 0.6983245764221752],
        [0.9091106954382898, 0.3847272671471867, -0.15969556460079362],
        [-0.1564703694572673, -0.13595070453226268, -0.9782813651598848],
        [0.2534426665477384, -0.8372648077317534, -0.4845146607762059],
        [0.3469539658355512, -0.09674802369360846, -0.9328787517691484],
        [0.6552649078301453, 0.7491382504989829, 0.09705556246642744],
        [-0.448438279665472, 0.8764756435845373, -0.17519576345831844],
        [-0.9678077676660033, -0.06894370579492914, -0.24206381447174172],
        [-0.9224247301219554, -0.041328486586272986, 0.38395907784023825],
        [-0.3251931532775624, 0.6335378580679545, 0.7020535559742322],
        [-0.32128909901008884, 0.800805664025917, 0.5054538587460734],
        [-0.8970641760427237, 0.09024553666897084, -0.43258710934575784],
        [0.889377511868408, 0.44827991764964875, 0.0897371540376909],
        [-0.5029630896787949, 0.22894231132496123, 0.8334347895942101],
        [0.9766248807371016, 0.212938186130377, 0.029345718814626352],
        [-0.07491688082598648, 0.030040455816011905, 0.9967371930361941],
        [0.19892783702785102, 0.6011881760597172, -0.773951221086582],
        [0.7342212342509201, -0.5009092255597697, -0.4582675276780663],
        [0.5716893348884035, 0.5748250149217536, 0.5854464165020234],
        [-0.0818930853778529, -0.9713540414227224, -0.22308036394770306],
        [0.16968612173400321, -0.9854378090199069, -0.010906174623522277],
        [0.3686113891575745, 0.7030274125614355, -0.6081760443905172],
        [0.5748944675712456, 0.12896769597339972, 0.8079998047965649],
        [-0.69253737071191, -0.46802413987897784, 0.5489493552760378],
        [0.5454783623779208, -0.8171601068689435, 0.1862866498690627],
        [-0.05314209215585929, -0.3672521703192491, -0.9286020468624326],
        [0.7797988043199621, 0.5348309005215345, -0.3253762938944421],
        [-0.7879662173745184, -0.04624308402824871, 0.6139794926999189],
        [-0.4552309354556275, -0.2818672358747745, 0.8445801659668007],
    ]),
}

In [ ]:
# @title Visualization of configurations

import numpy as np
import plotly.graph_objects as go

def normalize_and_plot_3d_plotly(points_array):
    """
    Normalizes a set of 3D points to have a norm of 1 and plots them on
    a 3D interactive scatter plot with a transparent unit sphere using Plotly.

    Args:
        points_array (np.ndarray): A NumPy array of shape [n, 3]
                                   representing n 3D points.

    Returns:
        go.Figure: The Plotly Figure object.
    """
    if points_array.ndim != 2 or points_array.shape[1] != 3:
        raise ValueError("Input array must be of shape [n, 3].")

    # 1. Normalize the points
    # Calculate the norm (length) of each vector (row)
    norms = np.linalg.norm(points_array, axis=1)

    # Use broadcasting to divide each point by its norm.
    # Add a small epsilon to prevent division by zero for points near the origin.
    epsilon = 1e-10
    normalized_points = points_array / (norms[:, np.newaxis] + epsilon)

    # 2. Generate points for the transparent unit sphere (center: (0,0,0), radius: 1)
    # Use spherical coordinates to create the mesh for the unit sphere
    r = 1  # Radius of the unit sphere
    phi = np.linspace(0, 2 * np.pi, 100) # Azimuthal angle
    theta = np.linspace(0, np.pi, 50)  # Polar angle

    # Create 2D grids of angles
    phi, theta = np.meshgrid(phi, theta)

    # Convert spherical to Cartesian coordinates
    x_sphere = r * np.sin(theta) * np.cos(phi)
    y_sphere = r * np.sin(theta) * np.sin(phi)
    z_sphere = r * np.cos(theta)

    # 3. Create the Plotly Figure
    fig = go.Figure()

    # Add the transparent unit sphere (as a Surface plot)
    fig.add_trace(go.Surface(
        x=x_sphere,
        y=y_sphere,
        z=z_sphere,
        colorscale='Blues',
        opacity=0.2,  # Set transparency
        showscale=False, # Hide the color scale legend for the sphere
        name='Unit Sphere'
    ))

    # Add the normalized points (as a Scatter3D plot)
    fig.add_trace(go.Scatter3d(
        x=normalized_points[:, 0],
        y=normalized_points[:, 1],
        z=normalized_points[:, 2],
        mode='markers',
        marker=dict(
            size=4,
            color='red',
            opacity=1
        ),
        name='Normalized Points'
    ))

    # 4. Set plot layout and aesthetics
    fig.update_layout(
        title='Normalized 3D Points on a Transparent Unit Sphere (Plotly)',
        scene=dict(
            # xaxis=dict(title='X Axis', range=[-1.1, 1.1]),
            # yaxis=dict(title='Y Axis', range=[-1.1, 1.1]),
            # zaxis=dict(title='Z Axis', range=[-1.1, 1.1]),
            xaxis=dict(range=[-1.1, 1.1]),
            yaxis=dict(range=[-1.1, 1.1]),
            zaxis=dict(range=[-1.1, 1.1]),
            # Ensure 3D aspect ratio is equal for a proper sphere shape
            aspectmode='cube',
        ),
        # Set background colors for a clean look
        scene_bgcolor='white',
        paper_bgcolor='white'
    )

    return fig

# Example Usage:

# Create some random 3D points
np.random.seed(42)
num_points = 100
# Generate points with varying magnitudes
points_data = np.random.randn(num_points, 3) * np.array([3, 1, 5])

# Run the function with the example data
# fig = normalize_and_plot_3d_plotly(icosahedron_vertices)
fig = normalize_and_plot_3d_plotly(tammes_results[50])

# To display the figure in a Jupyter Notebook, simply call the figure object
fig.show()

In [ ]:
# @title Programs obtained by AlphaEvolve


# Helper function for Fibonacci Sphere initialization
def _fibonacci_sphere(num_points: int) -> np.ndarray:
  """Generates num_points approximately uniformly on a unit sphere using Fibonacci sphere algorithm."""
  if num_points == 0:
    return np.empty((0, 3))
  if num_points == 1:
    return np.array(
        [[0.0, 0.0, 1.0]]
    )  # Place single point at North Pole for n=1

  points = np.zeros((num_points, 3))

  # Golden angle for spiral distribution
  golden_angle = np.pi * (3 - np.sqrt(5))

  for i in range(num_points):
    # Evenly distribute points along the z-axis, avoiding exact poles which can sometimes lead to numerical issues
    # and ensuring a symmetric distribution.
    z = 1 - (i * 2 + 1) / num_points  # z runs from approx. 1 to approx. -1

    radius = np.sqrt(1 - z * z)  # Radius of the circle at current z-height

    theta = golden_angle * i  # Azimuthal angle increment using the golden angle

    x = np.cos(theta) * radius
    y = np.sin(theta) * radius

    points[i] = [x, y, z]  # Store (x,y,z) coordinates
  return points


# Helper function to ensure points are on the unit sphere and handle zero vectors.
def _normalize_point_on_sphere(point: np.ndarray) -> np.ndarray:
  """Normalizes a 3D point to lie on the unit sphere.

  If the point is at the origin, it's moved to a random point on the sphere.
  """
  norm = np.linalg.norm(point)
  if (
      norm < 1e-12
  ):  # Using a small epsilon for robustness against near-zero norms
    new_point = np.random.randn(3)
    return new_point / np.linalg.norm(new_point)
  else:
    return point / norm


# Helper function to generate random points in a spherical cone.
def _generate_random_points_in_cone(
    num_points: int, cone_axis: np.ndarray, cone_angle: float
) -> np.ndarray:
  """Generates num_points uniformly random within a spherical cone on the unit sphere.

  Args:
      num_points: The number of points to generate.
      cone_axis: A 3D vector defining the center axis of the cone (will be
        normalized).
      cone_angle: The half-angle of the cone in radians (0 to pi).

  Returns:
      A numpy array of shape (num_points, 3) with points on the unit sphere.
  """
  if num_points == 0:
    return np.empty((0, 3))

  normalized_axis = _normalize_point_on_sphere(cone_axis)

  # Generate points in a canonical cone (axis is (0,0,1))
  z_min = np.cos(cone_angle)

  points_canonical = np.zeros((num_points, 3))

  for i in range(num_points):
    z = np.random.uniform(z_min, 1.0)
    phi = np.random.uniform(0.0, 2 * np.pi)

    # Calculate radius in the XY plane
    radius_xy = np.sqrt(1.0 - z * z)

    x = radius_xy * np.cos(phi)
    y = radius_xy * np.sin(phi)

    points_canonical[i] = [x, y, z]

  # Rotate canonical points to align with the desired cone_axis.
  # The canonical axis is (0,0,1).
  initial_pole = np.array([0.0, 0.0, 1.0])

  # Handle cases where target_direction (cone_axis) is antipodal to initial_pole
  if np.dot(initial_pole, normalized_axis) < -0.999:  # Nearly antipodal
    # Pick a random axis orthogonal to initial_pole
    axis = _normalize_point_on_sphere(np.random.randn(3))
    axis -= np.dot(axis, initial_pole) * initial_pole
    if np.linalg.norm(axis) < 1e-12:  # Fallback if random axis was parallel
      axis = np.array([0.0, 1.0, 0.0])
    axis = _normalize_point_on_sphere(axis)
    rotation = Rotation.from_rotvec(np.pi * axis)
  else:
    rotation = Rotation.align_vectors([normalized_axis], [initial_pole])[0]

  rotated_points = rotation.apply(points_canonical)

  # Final normalization for robustness
  return np.array([_normalize_point_on_sphere(p) for p in rotated_points])


# Helper function to perform a force-directed perturbation on a single random point.
def _single_point_force_perturbation(
    trial_positions: np.ndarray,
    n: int,
    current_perturb_scale: float,
    current_repulsion_power: float,
    min_distance_epsilon: float,
) -> None:
  """Performs a force-directed perturbation on a single random point."""
  random_index = np.random.randint(0, n)
  current_point_pos = trial_positions[random_index]

  diffs_from_rand_idx = current_point_pos - trial_positions
  dists = np.linalg.norm(diffs_from_rand_idx, axis=1)

  repulsion_exponent = current_repulsion_power + 1.0

  with np.errstate(divide="ignore", invalid="ignore"):
    inv_dists_pow = 1.0 / (
        np.power(dists, repulsion_exponent) + min_distance_epsilon
    )

  inv_dists_pow[random_index] = 0

  total_repulsion_for_move = np.sum(
      diffs_from_rand_idx * inv_dists_pow[:, np.newaxis], axis=0
  )

  radial_component = np.dot(total_repulsion_for_move, current_point_pos)
  tangential_force = (
      total_repulsion_for_move - radial_component * current_point_pos
  )

  norm_tangential_force = np.linalg.norm(tangential_force)

  perturb_vector = np.zeros(3)
  # Add a base perturbation even if force is zero to avoid stagnation, always tangential.
  random_dir_for_jitter = np.random.randn(3)
  tangent_random_jitter = (
      random_dir_for_jitter
      - np.dot(random_dir_for_jitter, current_point_pos) * current_point_pos
  )
  norm_tj = np.linalg.norm(tangent_random_jitter)

  if norm_tj < min_distance_epsilon:  # Fallback for robustness.
    # Pick a basis vector not parallel to current_point_pos
    v = current_point_pos
    basis1 = np.array([1.0, 0.0, 0.0])
    if np.abs(np.dot(v, basis1)) > 0.95:
      basis1 = np.array([0.0, 1.0, 0.0])
    tangent_random_jitter = np.cross(v, basis1)
    norm_tj = np.linalg.norm(tangent_random_jitter)  # Re-calculate norm

  perturb_direction = (
      tangent_random_jitter / norm_tj
  )  # Normalized tangential random direction.

  # Combine force-directed move with random jitter.
  if norm_tangential_force > min_distance_epsilon:
    perturb_vector = (
        tangential_force / norm_tangential_force * current_perturb_scale
    )
    # Add some random component to escape purely deterministic moves.
    perturb_vector += (
        perturb_direction * current_perturb_scale * 0.1
    )  # Small random tangential component
  else:
    # If no net force, just apply random tangential jitter.
    perturb_vector = (
        perturb_direction * current_perturb_scale * 0.5
    )  # A more significant random move.

  trial_positions[random_index] += perturb_vector
  trial_positions[random_index] = _normalize_point_on_sphere(
      trial_positions[random_index]
  )


def get_positions(n: int) -> np.ndarray:
  """Generate positions on the unit sphere to maximize the minimum pairwise distance."""
  variable_name = f"suggested_positions_{n}"

  # Known optimal configurations for small N. These are typically Platonic solids
  # vertices or highly symmetric arrangements known from Tammes problem literature.
  # Normalized after definition to ensure they are on the unit sphere.
  known_configs = {
      2: np.array([[1.0, 0.0, 0.0], [-1.0, 0.0, 0.0]]),  # Antipodal points
      3: np.array([
          [1.0, 0.0, 0.0],
          [-0.5, np.sqrt(3) / 2, 0.0],
          [-0.5, -np.sqrt(3) / 2, 0.0],
      ]),  # Equilateral triangle in equatorial plane
      4: np.array([
          [1.0, 1.0, 1.0],
          [1.0, -1.0, -1.0],
          [-1.0, 1.0, -1.0],
          [-1.0, -1.0, 1.0],
      ]),  # Tetrahedron vertices
      5: np.array([  # D_3h point group, trigonal bipyramid
          [0.0, 0.0, 1.0],  # North pole
          [0.0, 0.0, -1.0],  # South pole
          [1.0, 0.0, 0.0],  # Equator point 1
          [
              np.cos(2 * np.pi / 3),
              np.sin(2 * np.pi / 3),
              0.0,
          ],  # Equator point 2
          [
              np.cos(4 * np.pi / 3),
              np.sin(4 * np.pi / 3),
              0.0,
          ],  # Equator point 3
      ]),
      6: np.array([
          [1.0, 0.0, 0.0],
          [-1.0, 0.0, 0.0],
          [0.0, 1.0, 0.0],
          [0.0, -1.0, 0.0],
          [0.0, 0.0, 1.0],
          [0.0, 0.0, -1.0],
      ]),  # Octahedron vertices
      7: np.array([  # Optimal D_5h for N=7, min_dist = 1.2890656096001004 (from Packomania.com)
          [0.0, 0.0, 1.0],
          [0.920295, 0.0, 0.391216],
          [0.284279, 0.875691, 0.391216],
          [-0.744955, 0.536982, 0.391216],
          [-0.744955, -0.536982, 0.391216],
          [0.284279, -0.875691, 0.391216],
          [0.920295, 0.0, -0.739773],
      ]),
      8: (
          lambda: np.concatenate(
              (  # Optimal: Square antiprism. Based on previous good solution's formulas.
                  (
                      lambda r_v, h_v: np.array([
                          [r_v, 0.0, h_v],
                          [0.0, r_v, h_v],
                          [-r_v, 0.0, h_v],
                          [0.0, -r_v, h_v],
                      ])
                  )(
                      2.0 / np.sqrt(4.0 + np.sqrt(2.0)),
                      np.sqrt(np.sqrt(2.0) / (4.0 + np.sqrt(2.0))),
                  ),
                  (
                      lambda r_v, h_v: np.array([
                          [
                              r_v * np.cos(np.pi / 4.0),
                              r_v * np.sin(np.pi / 4.0),
                              -h_v,
                          ],
                          [
                              -r_v * np.cos(np.pi / 4.0),
                              r_v * np.sin(np.pi / 4.0),
                              -h_v,
                          ],
                          [
                              -r_v * np.cos(np.pi / 4.0),
                              -r_v * np.sin(np.pi / 4.0),
                              -h_v,
                          ],
                          [
                              r_v * np.cos(np.pi / 4.0),
                              -r_v * np.sin(np.pi / 4.0),
                              -h_v,
                          ],
                      ])
                  )(
                      2.0 / np.sqrt(4.0 + np.sqrt(2.0)),
                      np.sqrt(np.sqrt(2.0) / (4.0 + np.sqrt(2.0))),
                  ),
              ),
              axis=0,
          )
      )(),
      9: (
          lambda: np.concatenate(
              (  # Optimal D_3h for N=9, min_dist approx 1.15470
                  (
                      lambda r_eq, z_eq: np.array([  # Equatorial triangle
                          [
                              r_eq * np.cos(k * 2 * np.pi / 3),
                              r_eq * np.sin(k * 2 * np.pi / 3),
                              z_eq,
                          ]
                          for k in range(3)
                      ])
                  )(1.0, 0.0),
                  (
                      lambda r_up, z_up: np.array([  # Upper triangle
                          [
                              r_up * np.cos(k * 2 * np.pi / 3),
                              r_up * np.sin(k * 2 * np.pi / 3),
                              z_up,
                          ]
                          for k in range(3)
                      ])
                  )(np.sqrt(3) / 2, 0.5),
                  (
                      lambda r_down, z_down: np.array([  # Lower triangle, rotated
                          [
                              r_down * np.cos(k * 2 * np.pi / 3 + np.pi / 3),
                              r_down * np.sin(k * 2 * np.pi / 3 + np.pi / 3),
                              z_down,
                          ]
                          for k in range(3)
                      ])
                  )(np.sqrt(3) / 2, -0.5),
              ),
              axis=0,
          )
      )(),
      10: (
          lambda: np.concatenate(
              (  # Optimal D_5h for N=10, min_dist approx 1.09114
                  (
                      lambda z0_val: np.array([  # 5 upper points
                          [
                              np.sqrt(1 - z0_val**2)
                              * np.cos(k * 2 * np.pi / 5),
                              np.sqrt(1 - z0_val**2)
                              * np.sin(k * 2 * np.pi / 5),
                              z0_val,
                          ]
                          for k in range(5)
                      ])
                  )(0.443190),
                  (
                      lambda z1_val: np.array([  # 5 lower points, rotated
                          [
                              np.sqrt(1 - z1_val**2)
                              * np.cos(k * 2 * np.pi / 5 + np.pi / 5),
                              np.sqrt(1 - z1_val**2)
                              * np.sin(k * 2 * np.pi / 5 + np.pi / 5),
                              z1_val,
                          ]
                          for k in range(5)
                      ])
                  )(-0.812251),
              ),
              axis=0,
          )
      )(),
      11: (
          lambda: np.concatenate(
              (  # Optimal D_5h for N=11, min_dist approx 1.05146
                  np.array([[0.0, 0.0, 1.0]]),  # North pole
                  (
                      lambda z1_val: np.array([  # 5 upper points
                          [
                              np.sqrt(1 - z1_val**2)
                              * np.cos(k * 2 * np.pi / 5),
                              np.sqrt(1 - z1_val**2)
                              * np.sin(k * 2 * np.pi / 5),
                              z1_val,
                          ]
                          for k in range(5)
                      ])
                  )(0.457816),
                  (
                      lambda z2_val: np.array([  # 5 lower points, rotated
                          [
                              np.sqrt(1 - z2_val**2)
                              * np.cos(k * 2 * np.pi / 5 + np.pi / 5),
                              np.sqrt(1 - z2_val**2)
                              * np.sin(k * 2 * np.pi / 5 + np.pi / 5),
                              z2_val,
                          ]
                          for k in range(5)
                      ])
                  )(-0.748689),
              ),
              axis=0,
          )
      )(),
      12: np.array([  # Regular icosahedron using phi = (1+sqrt(5))/2
          [0.0, 1.0, (1.0 + np.sqrt(5.0)) / 2.0],
          [0.0, 1.0, -(1.0 + np.sqrt(5.0)) / 2.0],
          [0.0, -1.0, (1.0 + np.sqrt(5.0)) / 2.0],
          [0.0, -1.0, -(1.0 + np.sqrt(5.0)) / 2.0],
          [1.0, (1.0 + np.sqrt(5.0)) / 2.0, 0.0],
          [1.0, -(1.0 + np.sqrt(5.0)) / 2.0, 0.0],
          [-1.0, (1.0 + np.sqrt(5.0)) / 2.0, 0.0],
          [-1.0, -(1.0 + np.sqrt(5.0)) / 2.0, 0.0],
          [(1.0 + np.sqrt(5.0)) / 2.0, 0.0, 1.0],
          [(1.0 + np.sqrt(5.0)) / 2.0, 0.0, -1.0],
          [-(1.0 + np.sqrt(5.0)) / 2.0, 0.0, 1.0],
          [-(1.0 + np.sqrt(5.0)) / 2.0, 0.0, -1.0],
      ]),
      13: (
          lambda: np.concatenate(
              (  # Heuristic N=13: Icosahedron + North pole. Good starting point for SA.
                  np.array([  # Icosahedron vertices
                      [0.0, 1.0, (1.0 + np.sqrt(5.0)) / 2.0],
                      [0.0, 1.0, -(1.0 + np.sqrt(5.0)) / 2.0],
                      [0.0, -1.0, (1.0 + np.sqrt(5.0)) / 2.0],
                      [0.0, -1.0, -(1.0 + np.sqrt(5.0)) / 2.0],
                      [1.0, (1.0 + np.sqrt(5.0)) / 2.0, 0.0],
                      [1.0, -(1.0 + np.sqrt(5.0)) / 2.0, 0.0],
                      [-1.0, (1.0 + np.sqrt(5.0)) / 2.0, 0.0],
                      [-1.0, -(1.0 + np.sqrt(5.0)) / 2.0, 0.0],
                      [(1.0 + np.sqrt(5.0)) / 2.0, 0.0, 1.0],
                      [(1.0 + np.sqrt(5.0)) / 2.0, 0.0, -1.0],
                      [-(1.0 + np.sqrt(5.0)) / 2.0, 0.0, 1.0],
                      [-(1.0 + np.sqrt(5.0)) / 2.0, 0.0, -1.0],
                  ]),
                  np.array([[0.0, 0.0, 1.0]]),  # Add one point at North pole.
              ),
              axis=0,
          )
      )(),
      16: (
          lambda: np.concatenate(
              (  # Optimal D_4h for N=16, min_dist approx 0.88194
                  (
                      lambda z0_val: np.array([  # 4 upper-outer points
                          [
                              np.sqrt(1 - z0_val**2) * np.cos(k * np.pi / 2),
                              np.sqrt(1 - z0_val**2) * np.sin(k * np.pi / 2),
                              z0_val,
                          ]
                          for k in range(4)
                      ])
                  )(0.281898),
                  (
                      lambda z0_val: np.array([  # 4 upper-inner points, rotated
                          [
                              np.sqrt(1 - z0_val**2)
                              * np.cos(k * np.pi / 2 + np.pi / 4),
                              np.sqrt(1 - z0_val**2)
                              * np.sin(k * np.pi / 2 + np.pi / 4),
                              -z0_val,
                          ]
                          for k in range(4)
                      ])
                  )(0.281898),
                  (
                      lambda z1_val: np.array([  # 4 lower-outer points
                          [
                              np.sqrt(1 - z1_val**2) * np.cos(k * np.pi / 2),
                              np.sqrt(1 - z1_val**2) * np.sin(k * np.pi / 2),
                              z1_val,
                          ]
                          for k in range(4)
                      ])
                  )(-0.739773),
                  (
                      lambda z1_val: np.array([  # 4 lower-inner points, rotated
                          [
                              np.sqrt(1 - z1_val**2)
                              * np.cos(k * np.pi / 2 + np.pi / 4),
                              np.sqrt(1 - z1_val**2)
                              * np.sin(k * np.pi / 2 + np.pi / 4),
                              -z1_val,
                          ]
                          for k in range(4)
                      ])
                  )(-0.739773),
              ),
              axis=0,
          )
      )(),
  }

  # Normalize all hardcoded configurations to unit sphere after defining them.
  for key in known_configs:
    points_to_normalize = known_configs[key]
    if len(points_to_normalize) > 0:  # Ensure not empty before normalizing
      # Use helper for robustness.
      normalized_points = np.array(
          [_normalize_point_on_sphere(p) for p in points_to_normalize]
      )
      known_configs[key] = normalized_points

  # Determine initial positions based on a priority order.
  initial_positions = None

  # 1. Try to load a known good configuration from globals.
  if variable_name in globals():
    loaded_positions = globals()[variable_name]
    if loaded_positions.shape == (n, 3):
      # Use helper for robustness, handles n=0 case too.
      initial_positions = np.array(
          [_normalize_point_on_sphere(p) for p in loaded_positions]
      )

  # 2. If not loaded, use a hardcoded configuration for small n.
  if initial_positions is None and n in known_configs:
    initial_positions = known_configs[n]

  # 3. Otherwise, generate a Fibonacci sphere. This is a good general-purpose initializer.
  if initial_positions is None:
    initial_positions = _fibonacci_sphere(n)
    # _fibonacci_sphere is designed to generate points on the unit sphere,
    # so a global re-normalization is not strictly necessary here, but _normalize_point_on_sphere
    # will be applied later defensively if n > 0.

  # Handle n=0 explicitly for initial position.
  if n == 0:
    initial_positions = np.empty((0, 3))
    return initial_positions  # Early exit for n=0

  # Ensure the chosen initial configuration is normalized.
  # This final step is crucial if any previous step produced points that were slightly off-sphere.
  # Apply robust normalization to all points, using the helper function.
  initial_positions = np.array(
      [_normalize_point_on_sphere(p) for p in initial_positions]
  )

  best_positions = initial_positions.copy()
  curr_positions = initial_positions.copy()  # Current state for SA.
  best_score = get_score(best_positions)  # Initial global best score.
  curr_score = best_score  # Initial score of the current evolving state.

  # --- START OF Z-Axis M-Wave Initializer (Crazy Idea) ---
  if n > 1:  # Make sense only for multiple points
    # Randomly decide to apply this specific perturbation or not,
    # to introduce variability in initial guesses.
    apply_m_wave = np.random.rand() < 0.4  # Increased chance to 40%
    if apply_m_wave:
      # Small magnitude for the perturbation. Slightly increased.
      m_wave_strength = 0.008  # Base strength.
      # Randomly perturb either towards equator or poles.
      m_wave_direction = 1 if np.random.rand() < 0.5 else -1

      # Create a temporary array to apply perturbation
      temp_positions = best_positions.copy()

      # Compute current spherical coords' xy-plane radius
      radii_xy = np.linalg.norm(temp_positions[:, :2], axis=1)
      current_z = temp_positions[:, 2]

      # Perturb z coordinate using the "M-wave" pattern:
      # z * (1 - z^2) is zero at z=0, z=+-1 and peaks at z=+-1/sqrt(3).
      delta_z_perturb = (
          m_wave_strength * m_wave_direction * current_z * (1.0 - current_z**2)
      )
      temp_positions[:, 2] += delta_z_perturb

      # Re-normalize x,y to keep points on the sphere while preserving azimuthal angle.
      # Calculate the new radial distance in the xy-plane.
      # np.clip is used for robustness against floating point errors resulting in values slightly > 1 or < 0.
      new_radii_sq_safe = np.clip(1.0 - temp_positions[:, 2] ** 2, 1e-12, 1.0)
      new_radii_xy = np.sqrt(new_radii_sq_safe)

      # Avoid division by zero if original radii_xy was zero (point at pole).
      # If a point was at a pole (radii_xy close to 0), its x,y components become scaled by 1.0.
      with np.errstate(divide="ignore", invalid="ignore"):
        scale_xy = np.where(radii_xy > 1e-12, new_radii_xy / radii_xy, 1.0)

      temp_positions[:, 0] *= scale_xy
      temp_positions[:, 1] *= scale_xy

      # Final robust normalization for good measure.
      for i in range(n):
        temp_positions[i] = _normalize_point_on_sphere(temp_positions[i])

      # Evaluate this subtly perturbed initial state
      m_wave_score = get_score(temp_positions)

      # If this subtle perturbation improved the score, adopt it.
      if m_wave_score > best_score:
        logging.info(
            f"N={n}, Z-Axis M-Wave improved initial score from"
            f" {best_score:.3f} to {m_wave_score:.3f}"
        )
        best_positions = temp_positions.copy()
        curr_positions = temp_positions.copy()
        best_score = m_wave_score
        curr_score = m_wave_score
  # --- END OF Z-Axis M-Wave Initializer ---

  start_time = time.time()
  max_runtime = (
      290  # Allocate 290 seconds for the search, leaving a small buffer.
  )

  # Simulated Annealing parameters.
  initial_temp = 0.2  # Higher initial temp for broader exploration.
  final_temp = 5e-7  # Lower final temp for fine-grained exploitation.

  initial_perturb_scale = (
      0.05  # Stronger initial force for more aggressive moves.
  )
  min_perturb_scale = 5e-7  # Smaller minimum force for precise adjustments.

  # Repulsion exponent (P for force 1/r^P).
  # Standard inverse square (Coulomb-like) force, dynamically strengthening.
  initial_repulsion_power = 2.0
  final_repulsion_power = 3.0
  min_distance_epsilon = 1e-10  # Small constant for numerical stability.

  # Stagnation-triggered shake parameters for basin hopping.
  stagnation_threshold_iterations = max(50, int(n * 2))  # Dynamic threshold
  shake_proportion_of_points = (
      0.2  # How many points to re-randomize during shake
  )
  min_points_to_shake = 1
  global_rotation_prob = 0.25  # Probability of global rotation during a shake.

  stagnation_counter = 0  # Tracks accepted moves without global best.

  # The main optimization loop.
  while time.time() - start_time < max_runtime:
    elapsed_time = time.time() - start_time
    progress = elapsed_time / max_runtime

    # Dynamic parameter updates for Simulated Annealing and force-directed moves.
    # Exponential cooling for temperature and perturbation scale.
    temp = initial_temp * (final_temp / initial_temp) ** progress
    temp = max(temp, final_temp)

    current_perturb_scale = (
        initial_perturb_scale
        * (min_perturb_scale / initial_perturb_scale) ** progress
    )
    current_perturb_scale = max(current_perturb_scale, min_perturb_scale)

    current_repulsion_power = (
        initial_repulsion_power
        * (final_repulsion_power / initial_repulsion_power) ** progress
    )

    trial_positions = curr_positions.copy()

    # Propose a new state.
    if n == 0:
      pass  # No points to perturb.
    elif n == 1:
      # For n=1, only tangential random perturbation is meaningful.
      current_point_pos = trial_positions[0]
      random_dir = np.random.randn(3)
      # Project random_dir onto the tangent plane.
      tangent_random = (
          random_dir - np.dot(random_dir, current_point_pos) * current_point_pos
      )
      norm_tr = np.linalg.norm(tangent_random)
      if norm_tr > min_distance_epsilon:
        perturb_vector = tangent_random / norm_tr * current_perturb_scale
      else:  # Fallback: generate an orthogonal vector.
        v = current_point_pos
        # Pick a basis vector not parallel to v
        basis1 = np.array([1.0, 0.0, 0.0])
        if np.abs(np.dot(v, basis1)) > 0.95:
          basis1 = np.array([0.0, 1.0, 0.0])
        perturb_vector = np.cross(v, basis1)
        perturb_vector /= np.linalg.norm(perturb_vector)
        perturb_vector *= current_perturb_scale

      trial_positions[0] += perturb_vector
      trial_positions[0] = _normalize_point_on_sphere(trial_positions[0])
    else:  # n > 1
      _single_point_force_perturbation(
          trial_positions,
          n,
          current_perturb_scale,
          current_repulsion_power,
          min_distance_epsilon,
      )

    trial_score = get_score(trial_positions)

    # Simulated Annealing acceptance criterion:
    delta_score = trial_score - curr_score

    if delta_score > 0:
      curr_positions = trial_positions.copy()
      curr_score = trial_score
      if curr_score > best_score:
        best_positions = curr_positions.copy()
        best_score = curr_score
        stagnation_counter = 0  # Reset counter on global improvement.
      else:
        stagnation_counter += 1  # Accepted a state, but no new global best.
    elif temp > 1e-8 and np.random.rand() < np.exp(delta_score / temp):
      curr_positions = trial_positions.copy()
      curr_score = trial_score
      stagnation_counter += 1  # Even if accepted, still count towards stagnation if no global improvement.
    else:
      stagnation_counter += 1  # Reject: count towards stagnation.

    # Stagnation-Triggered Global Shake/Basin Hopping.
    if stagnation_counter >= stagnation_threshold_iterations and n > 1:
      logging.info(
          f"N={n}, Stagnation detected. Applying a shake (elapsed:"
          f" {elapsed_time:.1f}s, temp: {temp:.4f})."
      )

      shaken_positions = (
          best_positions.copy()
      )  # Always shake from the global best configuration.

      # Probability distribution for different shake types.
      # Added "Coordinated Cluster Perturbation" for local, coherent multi-point exploration.
      shake_type_probs = [
          0.10,
          0.15,
          0.10,
          0.10,
          0.15,
          0.15,
          0.25,
      ]  # Global Rotation, Gravitational Wave, Entangled Jiggle, Quantum Retile, Cone Reseed, Chaotic Jiggle, Coordinated Cluster Perturbation
      shake_type = np.random.choice(
          [
              "global_rot",
              "grav_wave",
              "entangled_jiggle",
              "quantum_retile",
              "cone_reseed",
              "chaotic_jiggle",
              "coordinated_cluster_perturb",
          ],
          p=shake_type_probs,
      )

      if shake_type == "global_rot":
        # Global rotation (Spherical Voodoo Dance)
        logging.info(f"N={n}, Applying global rotation.")
        random_rotation = Rotation.random()
        shaken_positions = random_rotation.apply(shaken_positions)
        # Ensure points are still on sphere after rotation due to potential float errors
        for i in range(n):
          shaken_positions[i] = _normalize_point_on_sphere(shaken_positions[i])
      elif shake_type == "grav_wave":
        # "Gravitational Wave" - Spherical Harmonic Perturbation
        logging.info(f"N={n}, Applying Gravitational Wave perturbation.")

        # Convert points to spherical coordinates for sph_harm.
        shaken_positions_norms = np.linalg.norm(shaken_positions, axis=1)
        # Clamp z/r to [-1, 1] to avoid arccos errors due to floating point inaccuracies
        z_over_r_clamped = np.clip(
            shaken_positions[:, 2] / shaken_positions_norms, -1.0, 1.0
        )
        phi_coords = np.arccos(z_over_r_clamped)  # Polar angle (colatitude)
        theta_coords = np.arctan2(
            shaken_positions[:, 1], shaken_positions[:, 0]
        )  # Azimuthal angle (longitude)

        # Choose a random spherical harmonic mode (l, m) for distortion.
        # Low orders (l=1 to 5) for smoother, broader distortions.
        l_rand = np.random.randint(1, 6)  # l = 1, 2, 3, 4, 5
        m_rand = np.random.randint(-l_rand, l_rand + 1)  # m ranges from -l to l

        # Calculate the spherical harmonic value for all points (real part for scalar perturbation).
        harmonic_values = scipy.special.sph_harm(
            m_rand, l_rand, theta_coords, phi_coords
        ).real

        # Scale for harmonic perturbation, annealing with temperature.
        grav_wave_magnitude = (
            0.5 * temp
        )  # Use temperature for annealing magnitude.

        # Apply perturbation radially, then re-normalize. This effectively creates tangential movement.
        shaken_positions += (harmonic_values * grav_wave_magnitude)[
            :, np.newaxis
        ] * shaken_positions

        # Re-normalize all points robustly.
        for i in range(n):
          shaken_positions[i] = _normalize_point_on_sphere(shaken_positions[i])

      elif shake_type == "entangled_jiggle":
        # "Quantum Entanglement Jiggle & Phase Shift"
        logging.info(
            f"N={n}, Applying Quantum Entanglement Jiggle & Phase Shift."
        )

        # Find the pair of points (p1_idx, p2_idx) that are closest to each other.
        dists_matrix = np.linalg.norm(
            shaken_positions[:, np.newaxis, :]
            - shaken_positions[np.newaxis, :, :],
            axis=2,
        )
        np.fill_diagonal(dists_matrix, np.inf)

        if n < 2:
          logging.warning(f"N={n} is too small for entangled jiggle. Skipping.")
          # Fallback to random subset, handled by the final 'else' if none of the specific types trigger.
        else:
          min_dist_idx = np.unravel_index(
              np.argmin(dists_matrix), dists_matrix.shape
          )
          p1_idx, p2_idx = min_dist_idx[0], min_dist_idx[1]

          num_core_neighbors = min(n - 2, 2)
          core_indices = set([p1_idx, p2_idx])

          dists_to_p1 = dists_matrix[p1_idx].copy()
          dists_to_p1[p2_idx] = np.inf
          closest_to_p1_indices = np.argsort(dists_to_p1)[:num_core_neighbors]
          core_indices.update(closest_to_p1_indices)

          dists_to_p2 = dists_matrix[p2_idx].copy()
          dists_to_p2[p1_idx] = np.inf
          closest_to_p2_indices = np.argsort(dists_to_p2)[:num_core_neighbors]
          core_indices.update(closest_to_p2_indices)

          core_indices_list = list(core_indices)
          random.shuffle(core_indices_list)

          random_shift_direction = _normalize_point_on_sphere(
              np.random.randn(3)
          )

          jiggle_magnitude = current_perturb_scale * 2.0

          for idx in core_indices_list:
            current_pos = shaken_positions[idx]

            dist_to_p1 = np.linalg.norm(current_pos - shaken_positions[p1_idx])
            dist_to_p2 = np.linalg.norm(current_pos - shaken_positions[p2_idx])

            if dist_to_p1 < dist_to_p2:
              tangential_shift = (
                  random_shift_direction
                  - np.dot(random_shift_direction, current_pos) * current_pos
              )
            else:
              tangential_shift = (
                  -random_shift_direction
                  - np.dot(-random_shift_direction, current_pos) * current_pos
              )

            norm_ts = np.linalg.norm(tangential_shift)
            if norm_ts > min_distance_epsilon:
              shaken_positions[idx] += jiggle_magnitude * (
                  tangential_shift / norm_ts
              )
            else:
              random_jitter = np.random.randn(3)
              tangent_jitter = (
                  random_jitter
                  - np.dot(random_jitter, current_pos) * current_pos
              )
              norm_tj = np.linalg.norm(tangent_jitter)
              if norm_tj > min_distance_epsilon:
                shaken_positions[idx] += jiggle_magnitude * (
                    tangent_jitter / norm_tj
                )

            shaken_positions[idx] = _normalize_point_on_sphere(
                shaken_positions[idx]
            )
      elif shake_type == "quantum_retile":
        # "Quantum Re-tiling" - replacing a subset of points with a structured tile.
        logging.info(f"N={n}, Applying Quantum Re-tiling.")

        k_retile = max(
            3, min(n - 1, int(n * 0.15))
        )  # Select 15% of points, min 3, max n-1 (to ensure some original points remain)
        if k_retile == 0:
          logging.info("N too small for quantum retile, skipping.")
        else:
          # 1. Randomly select k_retile points to be replaced.
          indices_to_replace = np.random.choice(n, size=k_retile, replace=False)

          # 2. Generate a "Quantum Tile" (e.g., a small Fibonacci sphere).
          new_tile_points = _fibonacci_sphere(k_retile)

          # 3. Random Orientation & Position for the tile.
          # Pick a random target direction (where the center of the tile will be pointed).
          target_direction = _normalize_point_on_sphere(np.random.randn(3))

          # The _fibonacci_sphere by default has its "pole" at (0,0,1).
          # Calculate the rotation to align (0,0,1) with target_direction.
          # Use 'align_vectors' for robust rotation calculation.
          if k_retile > 0:
            initial_pole = np.array([0.0, 0.0, 1.0])
            # Handle cases where target_direction is antipodal to initial_pole
            if (
                np.dot(initial_pole, target_direction) < -0.999
            ):  # Nearly antipodal
              # Pick a random axis orthogonal to initial_pole
              axis = _normalize_point_on_sphere(np.random.randn(3))
              axis -= np.dot(axis, initial_pole) * initial_pole
              if np.linalg.norm(axis) < min_distance_epsilon:
                axis = np.array([0.0, 1.0, 0.0])  # Fallback
              axis = _normalize_point_on_sphere(axis)
              rotation = Rotation.from_rotvec(np.pi * axis)
            else:
              rotation = Rotation.align_vectors(
                  [target_direction], [initial_pole]
              )[0]

            new_tile_points = rotation.apply(new_tile_points)

          # 4. Integrate: Replace the selected points with the new tile.
          shaken_positions[indices_to_replace] = new_tile_points

          # 5. Final normalization for all points (important for safety).
          for i in range(n):
            shaken_positions[i] = _normalize_point_on_sphere(
                shaken_positions[i]
            )

      elif shake_type == "cone_reseed":
        # "Conical Reseeding" - replace a subset of points with new points randomized within a cone.
        logging.info(f"N={n}, Applying Conical Reseeding.")

        k_reseed = max(
            3, min(n - 1, int(n * 0.15))
        )  # Select 15% of points, min 3, max n-1
        if k_reseed == 0:
          logging.info("N too small for conical reseed, skipping.")
        else:
          indices_to_replace = np.random.choice(n, size=k_reseed, replace=False)

          # Choose a random cone axis on the sphere.
          cone_axis = _normalize_point_on_sphere(np.random.randn(3))

          # Anneal the cone angle: wider at higher temps, narrower at lower temps.
          # E.g., from pi/2 (hemisphere) to pi/10 (tight cone).
          initial_cone_angle = np.pi / 2.0  # 90 degrees
          final_cone_angle = np.pi / 10.0  # 18 degrees
          current_cone_angle = (
              initial_cone_angle
              * (final_cone_angle / initial_cone_angle) ** progress
          )
          current_cone_angle = max(current_cone_angle, final_cone_angle)

          new_cone_points = _generate_random_points_in_cone(
              k_reseed, cone_axis, current_cone_angle
          )

          shaken_positions[indices_to_replace] = new_cone_points

          # Final normalization for all points.
          for i in range(n):
            shaken_positions[i] = _normalize_point_on_sphere(
                shaken_positions[i]
            )

      elif shake_type == "chaotic_jiggle":
        # "Chaotic Cluster Jiggle" - aggressively perturb a random subset of points.
        logging.info(f"N={n}, Applying Chaotic Cluster Jiggle.")

        num_points_to_jiggle = max(
            min_points_to_shake, int(n * 0.2)
        )  # Jiggle 20% of points, min 1
        if num_points_to_jiggle == 0:
          logging.info("N too small for chaotic jiggle, skipping.")
        else:
          indices_to_jiggle = np.random.choice(
              n, size=num_points_to_jiggle, replace=False
          )

          # Set a base jiggle magnitude, less aggressively annealed than local search perturb scale.
          # This ensures significant jumps even at low temperatures.
          jiggle_base_magnitude = (
              initial_perturb_scale * 0.5
          )  # Start at half of initial perturb scale
          jiggle_annealed_magnitude = jiggle_base_magnitude * np.sqrt(
              temp / initial_temp
          )  # Slower annealing (sqrt temp)
          jiggle_magnitude = max(
              jiggle_annealed_magnitude, min_perturb_scale * 5
          )  # Ensure it doesn't get too small

          for idx in indices_to_jiggle:
            current_pos = shaken_positions[idx]
            random_dir = np.random.randn(3)
            # Project random_dir onto the tangent plane.
            tangent_random = (
                random_dir - np.dot(random_dir, current_pos) * current_pos
            )
            norm_tr = np.linalg.norm(tangent_random)

            if norm_tr > min_distance_epsilon:
              perturb_vector = tangent_random / norm_tr * jiggle_magnitude
            else:  # Fallback: generate an orthogonal vector.
              v = current_pos
              basis1 = np.array([1.0, 0.0, 0.0])
              if np.abs(np.dot(v, basis1)) > 0.95:
                basis1 = np.array([0.0, 1.0, 0.0])
              perturb_vector = np.cross(v, basis1)
              perturb_vector /= np.linalg.norm(perturb_vector)
              perturb_vector *= jiggle_magnitude

            shaken_positions[idx] += perturb_vector
            shaken_positions[idx] = _normalize_point_on_sphere(
                shaken_positions[idx]
            )
      elif shake_type == "coordinated_cluster_perturb":
        # "Coordinated Cluster Perturbation" - perturb a local cluster of points coherently.
        logging.info(f"N={n}, Applying Coordinated Cluster Perturbation.")

        if (
            n < 3
        ):  # Need at least 3 points for a meaningful cluster (anchor + 2 neighbors).
          logging.warning(
              f"N={n} is too small for coordinated cluster perturb. Skipping."
          )
          # Fallback to general random subset handled by the final 'else'.
        else:
          # 1. Select a random anchor point.
          anchor_idx = np.random.randint(0, n)
          anchor_pos = shaken_positions[anchor_idx]

          # 2. Find k nearest neighbors to the anchor point.
          # k_neighbors: 2 to 10% of points, ensuring at least 2 neighbors for a meaningful cluster.
          num_to_sample = np.random.randint(2, max(3, int(n * 0.1) + 1))
          k_neighbors = min(n - 1, num_to_sample)

          # Calculate distances from anchor_pos to all other points.
          dists_from_anchor = np.linalg.norm(
              shaken_positions - anchor_pos, axis=1
          )
          dists_from_anchor[anchor_idx] = np.inf  # Exclude anchor itself.

          # Get indices of the k_neighbors closest points.
          neighbor_indices = np.argsort(dists_from_anchor)[:k_neighbors]

          cluster_indices = np.append(neighbor_indices, anchor_idx)

          # 3. Compute centroid of this cluster.
          cluster_points = shaken_positions[cluster_indices]
          cluster_centroid = np.mean(cluster_points, axis=0)
          cluster_centroid = _normalize_point_on_sphere(
              cluster_centroid
          )  # Project centroid to sphere.

          # Determine perturbation magnitude, annealed with temperature.
          perturb_magnitude = (
              current_perturb_scale * 5.0 * np.sqrt(temp / initial_temp)
          )
          perturb_magnitude = max(perturb_magnitude, min_perturb_scale * 5.0)

          # 4. Apply a random small rotation to the cluster *around the cluster centroid's axis*.
          if np.random.rand() < 0.7:  # High chance of rotation
            rot_axis = _normalize_point_on_sphere(np.random.randn(3))
            rot_angle = (
                np.random.uniform(-np.pi / 16, np.pi / 16)
                * perturb_magnitude
                * 2.0
            )  # Small angle, scaled

            # Ensure rotation axis is orthogonal to the cluster_centroid.
            # Project random_rot_axis onto the tangent plane of the cluster_centroid.
            rot_axis_tangent = (
                rot_axis - np.dot(rot_axis, cluster_centroid) * cluster_centroid
            )
            norm_rot_axis_tangent = np.linalg.norm(rot_axis_tangent)

            if (
                norm_rot_axis_tangent < min_distance_epsilon
            ):  # If random_rot_axis was parallel/anti-parallel
              # Generate a truly orthogonal vector to cluster_centroid.
              # Pick a basis vector not parallel to cluster_centroid.
              v = cluster_centroid
              basis1 = np.array([1.0, 0.0, 0.0])
              if np.abs(np.dot(v, basis1)) > 0.95:
                basis1 = np.array([0.0, 1.0, 0.0])
              rot_axis = np.cross(v, basis1)
            else:
              rot_axis = rot_axis_tangent / norm_rot_axis_tangent

            rot_axis = _normalize_point_on_sphere(
                rot_axis
            )  # Final normalization.

            rot_vec = rot_angle * rot_axis
            cluster_rotation = Rotation.from_rotvec(rot_vec)

            # Apply rotation relative to the cluster centroid.
            for idx in cluster_indices:
              point_relative_to_centroid = (
                  shaken_positions[idx] - cluster_centroid
              )
              rotated_point_relative = cluster_rotation.apply(
                  point_relative_to_centroid
              )
              shaken_positions[idx] = rotated_point_relative + cluster_centroid
              shaken_positions[idx] = _normalize_point_on_sphere(
                  shaken_positions[idx]
              )

          # 5. Apply a random tangential "jiggle" for the cluster.
          if np.random.rand() < 0.7:  # High chance of tangential jiggle
            jiggle_direction = _normalize_point_on_sphere(np.random.randn(3))

            # Project jiggle_direction onto the tangent plane of the cluster_centroid.
            tangent_jiggle = (
                jiggle_direction
                - np.dot(jiggle_direction, cluster_centroid) * cluster_centroid
            )
            norm_tj = np.linalg.norm(tangent_jiggle)
            if norm_tj > min_distance_epsilon:
              tangent_jiggle /= norm_tj
            else:
              # Fallback for numerical stability if tangent_jiggle is near zero.
              rand_vec = np.random.randn(3)
              tangent_jiggle = (
                  rand_vec
                  - np.dot(rand_vec, cluster_centroid) * cluster_centroid
              )
              tangent_jiggle /= (
                  np.linalg.norm(tangent_jiggle)
                  if np.linalg.norm(tangent_jiggle) > min_distance_epsilon
                  else np.array([0.0, 1.0, 0.0])
              )
              tangent_jiggle = _normalize_point_on_sphere(tangent_jiggle)

            # Apply a collective tangential shift to all cluster points.
            shift_magnitude = perturb_magnitude * np.random.uniform(
                0.1, 0.5
            )  # A fraction of perturb_magnitude

            for idx in cluster_indices:
              shaken_positions[idx] += tangent_jiggle * shift_magnitude
              shaken_positions[idx] = _normalize_point_on_sphere(
                  shaken_positions[idx]
              )

      shaken_score = get_score(shaken_positions)

      # Evaluate the shaken configuration with Metropolis criterion against `best_score`.
      # This allows jumps to potentially worse but escapable states.
      delta_shake_score = shaken_score - best_score

      if delta_shake_score > 0:
        # Always accept better global configurations.
        best_score = shaken_score
        best_positions = shaken_positions.copy()
        curr_positions = (
            shaken_positions.copy()
        )  # Continue evolution from this new better configuration.
        curr_score = shaken_score  # Update current state score.
        stagnation_counter = 0  # Reset counter on improvement.
      elif temp > 1e-8 and np.random.rand() < np.exp(delta_shake_score / temp):
        # Accept worse global configurations with a probability to jump out of local minima.
        curr_positions = shaken_positions.copy()  # Move to this state.
        curr_score = shaken_score  # Update current state score.
        stagnation_counter = (
            0  # Reset counter, as we've made a significant move.
        )
      else:
        # Reject the shake: revert `curr_positions` to `best_positions` to continue local search.
        curr_positions = best_positions.copy()
        curr_score = best_score  # Revert current state score.
        stagnation_counter = 0  # Reset counter.

  # Final defensive normalization before returning.
  if n == 0:
    return np.empty((0, 3))

  final_positions = np.array(
      [_normalize_point_on_sphere(p) for p in best_positions]
  )
  return final_positions


def get_positions(n: int) -> np.ndarray:
  """Generates n points on the unit sphere to maximize minimum pairwise distance."""
  variable_name = f"suggested_positions_{n}"
  best_positions = np.random.randn(n, 3)

  if np.random.rand() < 0.5 and variable_name in globals():
    best_positions = globals()[variable_name]

  # Ensure initial points are on the unit sphere.
  initial_positions = best_positions
  norms = np.linalg.norm(initial_positions, axis=1, keepdims=True)
  norms[norms == 0] = 1  # Avoid division by zero for null vectors.
  initial_positions /= norms

  best_positions = initial_positions.copy()
  best_score = get_score(best_positions)

  positions = initial_positions.copy()
  start_time = time.time()

  # Parameters for the force-directed simulation.
  force_power = (
      3.0  # Corresponds to 1/r^2 potential, strongly repels close points.
  )
  initial_step_size = 0.2
  iter_count = 0

  while time.time() - start_time < 295:  # Run for almost the full time.
    elapsed_time = time.time() - start_time
    # Linearly decreasing step size for convergence.
    step_size = initial_step_size * (1 - elapsed_time / 300.0)

    # Vectorized calculation of pairwise differences and distances.
    diffs = positions[:, np.newaxis, :] - positions[np.newaxis, :, :]
    dists = np.linalg.norm(diffs, axis=2)

    # Calculate repulsive forces.
    with np.errstate(divide="ignore"):
      inv_dists_pow = 1.0 / np.power(dists, force_power)
    np.fill_diagonal(inv_dists_pow, 0)
    inv_dists_pow[np.isinf(inv_dists_pow)] = 1e12  # Cap force for close points.

    forces = np.einsum("ij,ijk->ik", inv_dists_pow, diffs)

    # Project forces onto the tangent plane of the sphere.
    radial_components = np.sum(forces * positions, axis=1, keepdims=True)
    tangent_forces = forces - radial_components * positions

    # Update positions and project back to the sphere.
    positions += step_size * tangent_forces
    positions /= np.linalg.norm(positions, axis=1, keepdims=True)

    iter_count += 1
    # Check score periodically to save time from frequent get_score calls.
    check_interval = max(20, int(4000 / (n**2 + 1)))
    if iter_count % check_interval == 0:
      curr_score = get_score(positions)
      if curr_score > best_score:
        best_score = curr_score
        best_positions = positions.copy()

  # One final check with the last configuration.
  final_score = get_score(positions)
  if final_score > best_score:
    best_positions = positions.copy()

  return best_positions

In [ ]:
for num_pts, construction in tammes_results.items():
  print(f"Number of Points: {num_pts}")
  print(f"Energy: {get_energy(construction).energy}")